In [1]:
import requests, os
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader

In [2]:
# This is the archive token which can be obtained from https://ampel.zeuthen.desy.de/live/dashboard/tokens
# In order to retrieve ZTF partnership alerts your token needs to have the appropriate access
token = os.environ["ARCHIVE_TOKEN"]   # I have mine stored
header = {"Authorization": "bearer "+token}

In [3]:
endpoint_query = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'
endpoint_count = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/alerts/healpix/skymap/count'

In [4]:
query = {'jd': {'$gt': 2458728.479882375, '$lt': 2458731.479882375}, 'regions': [{'nside': 32, 'pixels': [8, 9, 30]}]} #, 'candidate': {'rb': {'$gt': 0.3}, 'magpsf': {'$gt': 15}, 'ndethist': {'$gt': 0, '$lte': 10}, 'jdstarthist': {'$gt': 2458728.479882375}}}

In [9]:
response_query = requests.post(endpoint_query, headers=header, json=query )
response_count = requests.post(endpoint_count, headers=header, json=query )

if not response_query.ok:
    print( 'Query creation failed.')

In [10]:
print(response_count.content)

print(response_query.next)

b'{"count":1025}'
None


In [7]:
response_query.json()

{'resume_token': 'yazuHznUd-nqErq2jUCI-Nalq88l82UjWW80MBlRPHY',
 'chunk_size': 100}

In [8]:
resume_token = response_query.json()['resume_token']
# The loader config contains the resume_token as stream identifier
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream":resume_token}


try:
    alertlist = []
    alertloader = ZTFArchiveAlertLoader(**config)
    alerts = alertloader.get_alerts()
    for alert in alerts:
        alertlist.append(alert)
        print(alert['objectId'], alert['candidate']['jd'])
except requests.exceptions.HTTPError as e:
    status_code = e.response.status_code
    if status_code==423:
        print('HTTP error {}: likely caused by server staging process. Wait and try again.'.format(status_code) )
    else:
        raise e
    
print('Found {} alerts'.format(len(alertlist)))


Found 0 alerts
